In [2]:
import numpy as np
import pandas as pd

MARTINS BRAGA Arthur
Yuri

#### PART 1 - CROSS-SECTION DATA

In [3]:
df = pd.read_csv('textfiles/HPRICE2.raw', delim_whitespace=True, header=None, names=[
                 'price', 'crime', 'nox', 'rooms', 'dist', 'radial', 'proptax', 'stratio', 'lowstat', 'lprice', 'lnox', 'lproptax'])
df.head()

,price,crime,nox,rooms,dist,radial,proptax,stratio,lowstat,lprice,lnox,lproptax
0,24000,0.006,5.38,6.57,4.09,1,29.6,15.3,4.98,10.085810,1.682688,5.690360
1,21599,0.027,4.69,6.42,4.97,2,24.2,17.8,9.14,9.980402,1.545433,5.488938
2,34700,0.027,4.69,7.18,4.97,2,24.2,17.8,4.03,10.454500,1.545433,5.488938
3,33400,0.032,4.58,7.00,6.06,3,22.2,18.7,2.94,10.416310,1.521699,5.402678
4,36199,0.069,4.58,7.15,6.06,3,22.2,18.7,5.33,10.496790,1.521699,5.402678


1. State the fundamental hypothesis under which the Ordinary Least Squares (OLS) estimators
   are unbiased.

The unobserved variable has a zero mean

Conditional mean = unconditional mean

X and u are uncorrelated

$$
E(u|X) = E(u) = 0
$$

2. Show that under this assumption the OLS estimators are indeed unbiased.

3. Explain the sample selection bias with an example from the course.

As discussed in the course, sample selection bias occurs when the sample used in analysis is not representative of the population under study.

For instance, during World War II, a sample of RAF planes returning from war zones was used to determine areas that needed reinforcement. However, this sample excluded planes that were shot down, resulting in a biased sample. Consequently, the conclusion that reinforcing areas with bullet holes was necessary was erroneous. In reality, other areas should have been reinforced since planes with bullet holes were still able to return, while others couldn't.

4. Explain the omitted variable bias with an example from the course


Omitted variable bias occurs when a relevant variable is left out of a statistical analysis, leading to biased and unreliable results.

In the example of the effect of fertilizers on agricultural yield in soybean cultivation seen in the course, the yield is influenced by various factors including climate and soil quality. When conducting an experiment by selecting plots of land and applying different amounts of fertilizer, there is a possibility of systematically applying more fertilizer to the plots with better soil quality. This leads to unreliable results due to the omission of the soil quality variable, which is an important factor in determining yield.

5. Explain the problem of multicollinearity. Is it a problem in this dataset?


Multicollinearity is a problem that occurs when two or more independent variables are highly correlated. This can lead to imprecise coefficient estimates and higher variances.

To determine if multicollinearity is a problem is this dataset, we calculate the correlation between some variables.

In [4]:
for i in range(2, 10):
    first_column = df.columns[i]
    for j in range(i + 1, 10):
        second_column = df.columns[j]
        corr = abs(np.corrcoef(df[first_column], df[second_column])[0, 1])
        if i != j and corr > 0.7:
            print(first_column, second_column, corr)


nox dist 0.7702224777713113
radial proptax 0.9102281885331848
lowstat lprice 0.7914385699194478


From the results above, this dataset contains some highly correlated coefficients, such as the $radial$ and the $proptax$ with a correlation of 0.91. Therefore, we can state that the multicollinearity is a problem in this dataset.

6. Create three categories of nox levels (low, medium, high), corresponding to the following percentiles: 0-25%, 26%-74%, 75%-100%


In [5]:
low, high = df['nox'].quantile([0.25, 0.75])

7. Compute for each category of nox level the average median price and comment on your results


8. Produce a scatter plot with the variable price on the y-axis and the variable nox on the x-axis. Is this a ceteris paribus effect?

9. Run a regression of price on a constant, crime, nox, rooms, proptax. Comment on the histogram of the residuals. Interpret all coefficients.

10. Run a regression of lprice on a constant, crime, nox, rooms, proptax. Comment on the histogram of the residuals. Interpret all coefficients.

11. Run a regression of lprice on a constant, crime, lnox, rooms, lproptax. Comment on the histogram of the residuals. Interpret all coefficients.

12. In the specification of question 10, test the hypothesis H0: nox <0 vs. H1: nox > 0 at the 1% level


13. In the specification of question 10, test the hypothesis H0: nox = 0 vs. H1: nox ≠ 0 at the 1% level using the p-value of the test


14. In the specification of question 10, test the hypothesis H0: crime = proptax at the 10% level

15. In the specification of question 10, test the hypothesis H0: nox = 0, proptax = 0 at the 10% level

16. In the specification of question 10, test the hypothesis H0: nox = -500, proptax = -100 at the 10% level using the p-value of the test

17. In the specification of question 10, test the hypothesis H0: nox + proptax = -1000 at the 10% level using the p-value of the test

18. In the specification of question 10, test the hypothesis that all coefficients are the same for observations with low levels of nox vs. medium and high levels of nox.

19. Repeat the test of question 18 but now assuming that only the coefficients of nox and proptax can change between the two groups of observations. State and test H0.
